# --- Day 8: Haunted Wasteland ---
You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about ghosts a few minutes ago.

One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of network of labeled nodes.

It seems like you're meant to use the left/right instructions to navigate the network. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: AAA and ZZZ. You feel like AAA is where you are now, and you have to follow the left/right instructions until you reach ZZZ.

This format defines each node of the network individually. For example:

`RL`

`AAA = (BBB, CCC)`

`BBB = (DDD, EEE)`

`CCC = (ZZZ, GGG)`

`DDD = (DDD, DDD)`

`EEE = (EEE, EEE)`

`GGG = (GGG, GGG)`

`ZZZ = (ZZZ, ZZZ)`

Starting with AAA, you need to look up the next element based on the next left/right instruction in your input. In this example, start with AAA and go right (R) by choosing the right element of AAA, CCC. Then, L means to choose the left element of CCC, ZZZ. By following the left/right instructions, you reach ZZZ in 2 steps.

Of course, you might not find ZZZ right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: RL really means RLRLRLRLRLRLRLRL... and so on. For example, here is a situation that takes 6 steps to reach ZZZ:

`LLR`

`AAA = (BBB, BBB)`

`BBB = (AAA, ZZZ)`

`ZZZ = (ZZZ, ZZZ)`

Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?

In [1]:
def getMap():
    with open("map.txt") as file:
        return file.read()

In [3]:
class Graph:
    """This graph will be used to represent the map"""
    
    def __init__(self):
        #Initiate the graph
        self.graph = dict()
        
    def addMapEdges(self, node, left=None, right=None):
        #This will add a node with a left and right neighbor specifically for the map input (Ex: AAA = (BBB, CCC))
        self.graph[node] = [left, right]
            
    def __str__(self):
        #Pretty much only used for testing to see the graph
        graphInfo = ""
        for node, neighbors in zip(self.graph.keys(), self.graph.values()):
            graphInfo += f"{node}: {neighbors}\n"
        return graphInfo
            
#Formatting
directions, mapNodes = getMap().split("\n\n")
originNodes = [x.split()[0] for x in mapNodes.split("\n")]
nodePairs = [x.split("(")[1] for x in mapNodes.split("\n")]
nodePairs = [x.replace(")", "") for x in nodePairs]
nodePairs = [x.split(", ") for x in nodePairs]

#Creating a graph to represent the map
#Each node has two neighbors kept in a list. The first item in the list is the left node, the second is the right node
maps = Graph()

#Adding edges to the map
for i in range(len(originNodes)):
    maps.addMapEdges(originNodes[i], nodePairs[i][0], nodePairs[i][1])
    
currentNode = "AAA"
numSteps = 0
#Repeat until we find the end node ("ZZZ")
while currentNode != "ZZZ":
    
    for direction in directions:
        if direction == "L": #Go left
            currentNode = maps.graph[currentNode][0]
        elif direction == "R": #Go right
            currentNode = maps.graph[currentNode][1]
        numSteps += 1
            
        #If the current node is "ZZZ break out of the loop"
        if currentNode == "ZZZ":
            break
            
print(f"Number of steps: {numSteps}")

Number of steps: 11911


# --- Part Two ---
The sandstorm is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take significantly more steps to escape!

What if the map isn't for people - what if the map is for ghosts? Are ghosts even bound by the laws of spacetime? Only one way to find out.

After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in A is equal to the number ending in Z! If you were a ghost, you'd probably just start at every node that ends with A and follow all of the paths at the same time until they all simultaneously end up at nodes that end with Z.

For example:

`LR`

`11A = (11B, XXX)`

`11B = (XXX, 11Z)`

`11Z = (11B, XXX)`

`22A = (22B, XXX)`

`22B = (22C, 22C)`

`22C = (22Z, 22Z)`

`22Z = (22B, 22B)`

`XXX = (XXX, XXX)`

Here, there are two starting nodes, 11A and 22A (because they both end with A). As you follow each left/right instruction, use that instruction to simultaneously navigate away from both nodes you're currently on. Repeat this process until all of the nodes you're currently on end with Z. (If only some of the nodes you're on end with Z, they act like any other node and you continue as normal.) In this example, you would proceed as follows:

- Step 0: You are at 11A and 22A.
- Step 1: You choose all of the left paths, leading you to 11B and 22B.
- Step 2: You choose all of the right paths, leading you to 11Z and 22C.
- Step 3: You choose all of the left paths, leading you to 11B and 22Z.
- Step 4: You choose all of the right paths, leading you to 11Z and 22B.
- Step 5: You choose all of the left paths, leading you to 11B and 22C.
- Step 6: You choose all of the right paths, leading you to 11Z and 22Z.

So, in this example, you end up entirely on nodes that end in Z after 6 steps.

Simultaneously start on every node that ends with A. How many steps does it take before you're only on nodes that end with Z?

In [5]:
import math

class Graph:
    """This graph will be used to represent the map"""
    
    def __init__(self):
        #Initiate the graph
        self.graph = dict()
        
    def addMapEdges(self, node, left=None, right=None):
        #This will add a node with a left and right neighbor specifically for the map input (Ex: AAA = (BBB, CCC))
        self.graph[node] = [left, right]
            
    def __str__(self):
        #Pretty much only used for testing to see the graph
        graphInfo = ""
        for node, neighbors in zip(self.graph.keys(), self.graph.values()):
            graphInfo += f"{node}: {neighbors}\n"
        return graphInfo
            
#Formatting
directions, mapNodes = getMap().split("\n\n")
originNodes = [x.split()[0] for x in mapNodes.split("\n")]
nodePairs = [x.split("(")[1] for x in mapNodes.split("\n")]
nodePairs = [x.replace(")", "") for x in nodePairs]
nodePairs = [x.split(", ") for x in nodePairs]

#Creating a graph to represent the map
#Each node has two neighbors kept in a list. The first item in the list is the left node, the second is the right node
maps = Graph()

#Adding edges to the map
for i in range(len(originNodes)):
    maps.addMapEdges(originNodes[i], nodePairs[i][0], nodePairs[i][1])
    
#Starts with all nodes ending in 'A'
currentNodes = startNodes = [x for x in originNodes if x[-1] == "A"]
allMinPaths = [] #Keeps track of the steps it takes for each start node to get to a node that ends in 'Z'
numSteps = 0

#Keep going while there are nodes that do not end in 'Z'
while currentNodes:
    
    #Loop through all directions
    for direction in directions:
        if direction == "L": #Go left for all currentNodes
            currentNodes = [maps.graph[x][0] for x in currentNodes]
        elif direction == "R": #Go right for all currentNodes
            currentNodes = [maps.graph[x][1] for x in currentNodes]
        numSteps += 1
            
        #If there is a node that ends in 'Z'
        if [x for x in currentNodes if x[-1] == "Z"]:
            allMinPaths.append(numSteps)
            currentNodes = [x for x in currentNodes if x[-1] != "Z"]
            
#Get the least common multiple of all steps
lcm = math.lcm(*allMinPaths)
            
print(f"Number of steps: {lcm}")
print(f"Number of steps: {lcm:,}")

Number of steps: 10151663816849
Number of steps: 10,151,663,816,849


# This will never finish running! Do not attempt to run!
(Although I do think it's a cool solution)

In [ ]:
class Graph:
    """This graph will be used to represent the map"""
    
    def __init__(self):
        #Initiate the graph
        self.graph = dict()
        
    def addMapEdges(self, node, left=None, right=None):
        #This will add a node with a left and right neighbor specifically for the map input (Ex: AAA = (BBB, CCC))
        self.graph[node] = [left, right]
            
    def __str__(self):
        #Pretty much only used for testing to see the graph
        graphInfo = ""
        for node, neighbors in zip(self.graph.keys(), self.graph.values()):
            graphInfo += f"{node}: {neighbors}\n"
        return graphInfo
            
#Formatting
directions, mapNodes = getMap().split("\n\n")
originNodes = [x.split()[0] for x in mapNodes.split("\n")]
nodePairs = [x.split("(")[1] for x in mapNodes.split("\n")]
nodePairs = [x.replace(")", "") for x in nodePairs]
nodePairs = [x.split(", ") for x in nodePairs]

#Creating a graph to represent the map
#Each node has two neighbors kept in a list. The first item in the list is the left node, the second is the right node
maps = Graph()

#Adding edges to the map
for i in range(len(originNodes)):
    maps.addMapEdges(originNodes[i], nodePairs[i][0], nodePairs[i][1])
    
#Starts with all nodes ending in 'A'
currentNodes = startNodes = [x for x in originNodes if x[-1] == "A"]
numSteps = 0

#Keep going while there are nodes that do not end in 'Z'
while [x for x in currentNodes if x[-1] != "Z"]:
    
    #Loop through all directions
    for direction in directions:
        if direction == "L": #Go left for all currentNodes
            currentNodes = [maps.graph[x][0] for x in currentNodes]
        elif direction == "R": #Go right for all currentNodes
            currentNodes = [maps.graph[x][1] for x in currentNodes]
        numSteps += 1
            
        #Check to see if all nodes end in 'Z'
        if [x for x in currentNodes if x[-1] == "Z"] == currentNodes:
            break

print(f"Number of steps: {numSteps}")